3 Tree based classification
-------------------

**Decision Tree** is the most widely used **additive** model for **supervised learning** that combines simple classification nodes for larger scale tasks of both **classification** and **regression**. In most cases, the basic element in the **decision tree** is a binary node that split the data further into two parts. In a tree, many of the nodes would split the data incrementally along the tree, and bring the final classes at a node (usually referred as the leaf) where the data would not be split any more. There are also cases that the data can be split into more than two classes as opposed to binary classes, such as the [**CHAID**](https://en.wikipedia.org/wiki/Chi-square_automatic_interaction_detection), which will not be covered in this session, but please do feel free to explore further through the link.

The logic of **additivity** can also be applied to **decision trees** to form an **ensemble model** call the **random forest** for **ensemble learning**. In **ensemble learning**, each tree is applied to the data and trained in same manner. The results of all the trees would then be aggregated, e.g. through averaging, to produce the final result. The **ensembled trees** will be covered later in this course, but not in this session. 

In this session, we will be focusing on **decision tree** and its application to the real GIS and Remote Sensing data that we have used in the previous sessions. The structure will be:

- 3.0 Decision tree based classification of real world GIS and Remote Sensing datasets
- 3.1 Further inspection: features and model configurations

<font color='blue'> ***Throughout the notebook, you will encounter Questions, highlighted in blue. Please feel free to discuss them with your classmates and advisors***</font>

<font color='green'> ***Throughout the notebook, you will also encounter shorter and longer Exercises, highlighted in green.***</font>

### 3.0 Decision tree based classification of real world GIS and Remote Sensing datasets.

Again, we will use the same dataset of the satellite images along with the labelled LULC types of the Netherlands. The labelled LULC types in the AOIs will be split into **training** and **test** datasets. The trained model will be applied to the larger area covering almost the entire Netherlands.

In [ ]:
%matplotlib inline  # In order to plot figures inline in Jupyter Notebook, we need to run this. But please ignore this in Colab.

In [ ]:
# Before reading the data we need to first clone the data on Github to our Colab workspace
!git clone https://github.com/jonwangio/uu_ml

In [ ]:
# Same real world dataset as you have already encountered in the previous sessions.
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
% pip install geopandas
import geopandas as gpd
% pip install rasterio
import rasterio
from rasterio.plot import show
from sklearn.svm import SVC

# The data contains sample LULC areas around dutch provinces North Holland and Utrecht.
aoi = gpd.read_file('uu_ml/data/aoi_NL_5_classes.shp')

print (aoi.head())

In [ ]:
# Below is a visualization of the sample LULC areas superimposed on a satellite image of part of the Netherlands
file_location = 'uu_ml/data/b5_2015.TIF'
b5_2020 = rasterio.open(file_location, nodata=0)

# We also prepare the color codes for visualization
colors = [(257, 71, 27), (98, 93, 78), (14, 79, 58), (26, 0, 50), (75, 90, 85), (347, 72, 60), (246, 79, 60)]
cols = []
for col in colors:
    pal = sns.light_palette(col, input="husl", n_colors=4)
    for rgb in pal[1:]:
        cols.append(rgb)

# A preview of color codes. Please delete the triple quotation marks to run the code.

fig, ax = plt.subplots(figsize=(20, 5))
for i, c in enumerate(cols):
    ax.add_artist(plt.Circle((i, 0), 0.4, color=c))
    plt.text(i, -1, i, horizontalalignment='center')
    ax.set_axis_off()
    ax.set_aspect(1)
    ax.autoscale()
    plt.xlim(-1.25,43.25)
    plt.ylim(-1,1)


# Assign color codes to LULC types 
symbology = {'Agriculture': cols[5],
             'Clear water': cols[20],
             'Deciduous forest': cols[13],
             'Residential': cols[17],
             'Sand': cols[11]}

# Visualize
fig,ax = plt.subplots(1,1, figsize=(10,10))
show(b5_2020, ax=ax, cmap='gray', alpha=0.25)
aoi.plot(ax=ax, column='land_cover', legend=True, color=aoi['land_cover'].map(symbology))

from matplotlib.lines import Line2D
custom_points = [Line2D([0], [0], marker="o", linestyle="none", markersize=5, color=color) for color in symbology.values()]
leg_points = ax.legend(custom_points, symbology.keys(), loc='upper right', frameon=False)
ax.add_artist(leg_points)

As you may recall, we will prepare the **training** and **test** datasets from the sample LULC types in AOIs. We will use the two bands of satellite images as the inputs and the manually delineated LULC types in the AOIs as the output labels. Did you still remember how we stacked the data together and extract the input band pixels along with labels within the AOIs?

In [ ]:
# Import necessary modules and functionalities

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imread
from sklearn.svm import SVC

# As before, we start with loading and stacking the image bands.
# Again, we start with 2 bands that appeared to be useful for the small dataset.
file_list = ['uu_ml/data/b5_2015.TIF', 'uu_ml/data/b6_2015.TIF']  # List to store file names

# You can print to see how the file_list looks like
print(file_list)

# Read the files and stack them together by calling their names
# Use the 'for' loop to iterate over the names to read files
stack = np.array([])  # Empty array to store the stacked images
for file in file_list:
    img = imread(file)  # Read each image file
    print(img.shape)  # Each time, also check the size of the image
    
    # In order to do clustering, image should be reshaped into a single column
    img_col = img.reshape(-1, 1)
    
    # Each time put the reshaped image into the stack
    stack = np.hstack((stack,img_col)) if stack.size else img_col
    # Also to check the size of the stack
    print(stack.shape)

In [ ]:
# We also need to rasterize our manually delineated LULC types in the AOIs as we did in previous sessions
# Use the rasterio again to rasterize the *.shp file

from rasterio import features
import pandas as pd

# Labels from the AOIs
aoi = gpd.read_file('uu_ml/data/aoi_NL_5_classes.shp')
aoi['aoi_cat'] = pd.Categorical(aoi['class'])

# Rasterize
rst = rasterio.open('uu_ml/data/b5_2015.TIF')  # Base image to rasterize the *.shp
meta = rst.meta.copy()  # Copy metadata from the base image
meta.update(compress='lzw')

# Burn the AOIs *.shp file into raster and save it
out_rst = 'uu_ml/data/aoi_rasterized.tif'
with rasterio.open(out_rst, 'w+', **meta) as out:
    out_arr = out.read(1)

    # Create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(aoi.geometry, aoi.aoi_cat))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)

In [ ]:
# we will further stack the rasterized labels from the AOIs with the input bands, and only focus on the pixels within the AOIs.
# The training and test datasets will be prepared from the pixels and labels within the AOIs

# Load the rasterized LULC types in the AOI and concatenate it together with the images

aoi_rst = rasterio.open('uu_ml/data/aoi_rasterized.tif').read(1)

# Stack the label with the input bands
data = np.c_[stack, aoi_rst.reshape(-1,)]

# Of course, we are only interested in pixels with LULC type labelled
data = data[np.where(data[:,data.shape[1]-1]!=0)]

# Recall our function for preparing training and test datasets.
# This time we re-write it a little bit to let the users of the function to split the data into training and test sets.

def trainTestSplit(x, y, training_proportion):
    data = np.c_[x, y]
    np.random.shuffle(data)  # Shuffle the data so that LULC types can spread over training and test sets
    x_train = data[:int(training_proportion*len(data)), :2]  # 70% of data for training
    x_test = data[int(training_proportion*len(data)):, :2]  # 30% for testing
    y_train = data[:int(training_proportion*len(data)), 2:].reshape(-1,)  # 70% of data for training
    y_test = data[int(training_proportion*len(data)):, 2:].reshape(-1,)  # 30% for testing
    return x_train, y_train, x_test, y_test

# This time, use a very small proportion of the data for training, say, 30%.
X_train, Y_train, X_test, Y_test = trainTestSplit(data[:,:-1], data[:,-1], 0.3)

In [ ]:
# Visualize the training and test datasets

from matplotlib.colors import ListedColormap

# Assign color codes to LULC types 
symbology2 = {11: cols[20],
              21: cols[11],
              31: cols[17],
              52: cols[5],
              61: cols[13]}

cm = ListedColormap(symbology2.values())
imin = min(symbology2)  # Colormap range
imax = max(symbology2)

# Visualize
classes = ['Clear water', 'Sand', 'Residential', 'Deciduous forest', 'Agriculture']

fig1,(ax1, ax2) = plt.subplots(1,2, figsize=(20,10))

scatter1 = ax1.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, cmap=cm, vmin=imin, vmax=imax, label='LULC types')
ax1.set_title('Training data')
ax1.legend(handles=scatter1.legend_elements()[0], labels=classes)

ax2.scatter(X_test[:, 0], X_test[:, 1], c=Y_test, cmap=cm, vmin=imin, vmax=imax, label='LULC types')
ax2.set_title('Test data')
ax2.legend(handles=scatter1.legend_elements()[0], labels=classes)

Now we have both the **training** and **test** datasets, where the inputs are pixel values from the two bands (2-dimensional data points), and the outputs are categorical LULC types (encoded as numerical numbers).

Given the fact that, as you already saw in previous sessions, the dataset we have is relatively simple to classify in the 2-dimensional feature space formed by the two bands, we will first try to inspect the **decision tree** with individual band, and see the model performance with only one band.

Try to interpret the visualization in terms of values for splits, number of samples in each split, how the split compared to the patterns in the scatterplots in the 2-dimensional feature space above? And where are the problems?

In [ ]:
# Import sklearn along with all necessary modules

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score,mean_squared_error

# Initiate a tree model
tree_depth = 3
model_tree = DecisionTreeClassifier(max_depth=tree_depth)

# Fit the model to your data.
# Please note that the output of this fitting is a model with several parameters that are configurable, so far you only configured "max_depth" while training/fitting.
model_tree.fit(X_train[:,0].reshape(-1,1), Y_train)

# Visualize split
fig = plt.figure(figsize=(tree_depth*4,tree_depth*3))
tree.plot_tree(model_tree, filled=True)

<font color='blue'>***Question 0.1: Please interpret the information at the nodes. For instance, what does the gini mean? What are the values in shown in the squared brackets? How the split sample numbers correspond to the data points shown in the feature space above?***</font>

Further inspect the predictive performance by using the **confusion matrix** along with more detailed accuracy metrics.

In [ ]:
Y_pred = pd.Series(list(model_tree.predict(X_test[:,0].reshape(-1,1))), name='DT prediction')  # Store the predicted value in Y_pred
Y_actu = pd.Series(list(Y_test), name='Manual delineation')

# Map the LULC codes to the actual name of LULC types

# First we need a mapping from the LULC codes to the actual LULC type name.
code_lulc = { 52: 'Agriculture',
              11: 'Clear water',
              61: 'Deciduous forest',
              31: 'Residential',
              21: 'Sand'}

# Now replace the non-intuitive numbers with actual LULC type names and store them into new variables
Y_actu2 = Y_actu.replace(code_lulc)
Y_pred2 = Y_pred.replace(code_lulc)

# Show the LULC coded confusion matrix
df_confusion2 = pd.crosstab(Y_actu2, Y_pred2)
df_confusion2

<font color='blue'>***Question 0.2: Now, do the accuracy metrics capture your visual comparison between the decision tree and the scatterplots in the feature space above. Which classes suffer the most from misclassification? And why?***</font>

In [ ]:
# Print out more detailed accuracy assessment report

from sklearn.metrics import classification_report

print(classification_report(Y_actu2, Y_pred2))

### 3.1 Further inspection: features and model configurations

Now you may want to try both bands in the sample datasets, and experiment with model configurations. Please pay attention to how the splits are shown with two features (bands). And please do modify the model configuration such as the ***tree depth***.

In [ ]:
from sklearn import tree

# Initiate a tree model
tree_depth = 3
model_tree = DecisionTreeClassifier(max_depth=tree_depth)

# Fit the model to your data.
# Please note that the output of this fitting is a model with several parameters that are configurable, so far you only configured "max_depth" while training/fitting.
model_tree.fit(X_train, Y_train)

# Visualize split
fig = plt.figure(figsize=(tree_depth*4,tree_depth*3))
tree.plot_tree(model_tree, filled=True)

<font color='blue'>***Question 0.3: You may notice that at each node, the data has been split by using one of the two features scripted as either [0] or [1]. Does the split value corresponds to the pattern in the scatterplot above?***</font>

In [ ]:
# Again, make predictions and compare with the known labels.

Y_pred = pd.Series(list(model_tree.predict(X_test)), name='DT prediction')  # Store the predicted value in Y_pred
Y_actu = pd.Series(list(Y_test), name='Manual delineation')

# Now replace the non-intuitive numbers with actual LULC type names and store them into new variables
Y_actu2 = Y_actu.replace(code_lulc)
Y_pred2 = Y_pred.replace(code_lulc)

# Show the LULC coded confusion matrix
df_confusion2 = pd.crosstab(Y_actu2, Y_pred2)
df_confusion2

In [ ]:
# Print out more detailed accuracy assessment report

from sklearn.metrics import classification_report

report=classification_report(Y_actu2, Y_pred2,output_dict=True)

rp = pd.DataFrame(report).transpose()
rp

<font color='green'>***Exercise 0.1: Please try attach precison and recall as extra row and column to the confusion matrix above to create a more coherent summary of the accuracy assessment.***</font>

If you do have a lot of **features** of the input *x* in the form of *(x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>,..., x<sub>n</sub>)*, you may soon get lost about how the data is split in the feature space and what the values mean in the tree nodes. However, there are many options to visualize the data splits along the tree. 

At the same time, you have already seen that some data value are better split along specific feature/dimension. It means that certain data point of a dimension or feature can be used best for capture data variance. Hence it is possible to rank feature importance in classification. 

In [ ]:
# The tree can also be visualized in a different way to see how data has been splitted.
# Let's plot how the features split the data

# Plot parameters
plot_colors = "ryb"
plot_step = 100

# Plot the decision boundary
plt.figure(figsize=(6, 6))

x_min, x_max = X_test[:, 0].min() - 1000, X_test[:, 0].max() + 1000
y_min, y_max = X_test[:, 1].min() - 1000, X_test[:, 1].max() + 1000
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                     np.arange(y_min, y_max, plot_step))
plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

Z = model_tree.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Blues)

plt.xlabel('feature_1')
plt.ylabel('feature_2')

plt.scatter(X_test[:, 0], X_test[:, 1], c=Y_test.reshape(Y_test.shape[0]), cmap='Oranges', edgecolor='black', s=45)

<font color='blue'>***Question 0.4: How will you interpret the usefulness of the features in classifying the data?***</font>

<font color='green'>***Exercise 0.2: How is the classification? Which way of visualization do you prefer? Please try to configure your model with different depths of trees and inspect how that impact the data split by using one of the above visualization option.***</font>

In [ ]:
# Get feature importance

importance = model_tree.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
    
# Plot feature importance
plt.bar([ind for ind in range(len(importance))], importance)
plt.show()

<font color='blue'>***Question 0.5: Does the ranking capture the usefulness of the feature shown above?***</font>

Finally, once you are satisfied with the **training** and **testing**, use the model to predict on the larger area.

In [ ]:
# Predict on the image stack
# In this case, the more powerful non-linear kernel model is used

Y_pred_all = model_tree.predict(stack)

In [ ]:
# Assign color codes to LULC types 
symbology = {'Agriculture': cols[5],
             'Clear water': cols[20],
             'Deciduous forest': cols[3],
             'Residential': cols[17],
             'Sand': cols[11]}

# Visualize
fig1,(ax1, ax2) = plt.subplots(1,2, figsize=(20,10))
show(b5_2020, ax=ax1, cmap='gray', alpha=0.25)
aoi.plot(ax=ax1, column='land_cover', legend=True, color=aoi['land_cover'].map(symbology))

from matplotlib.lines import Line2D
custom_points = [Line2D([0], [0], marker="o", linestyle="none", markersize=5, color=color) for color in symbology.values()]
leg_points = ax1.legend(custom_points, symbology.keys(), loc='upper right', frameon=False)
ax1.add_artist(leg_points)

# Assign color codes to LULC types 
symbology2 = {31: cols[17],
              52: cols[5],
              11: cols[20],
              21: cols[11],
              61: cols[3]}

# Visualize
# Because the predicted labels are still in one column, you need to reshape it back to original image shape
row, col = img.shape  # Get the original dimensions of the image
imin = min(symbology2)  # Colormap range
imax = max(symbology2)

print('Printing large image takes time...')
ax2.imshow(Y_pred_all.reshape(row, col), cmap=cm, interpolation='none', vmin=imin, vmax=imax)

We can further save the predicted raster map into a georeferenced TIFF, so that we can inspect it along with other data or maps. Please feel free to drag the save TIFF file into other free and open-source software (FOSS) for further inspection.

In [ ]:
# Again we use our image data for georeferencing information
rst = rasterio.open('uu_ml/data/b5_2015.TIF')  # Base image to rasterize the *.shp
meta = rst.meta.copy()  # Copy metadata from the base image
meta.update(compress='lzw')

# Burn the AOIs *.shp file into raster and save it
out_rst = 'uu_ml/data/tree_prediction.tif'
out_file = rasterio.open(
    out_rst,
    'w',
    driver='GTiff',
    height=row,
    width=col,
    count=1,
    dtype=Y_pred_all.dtype,
    crs=rst.crs,
    transform=rst.transform)

out_file.write(Y_pred_all.reshape(row, col),1)
out_file.close()

<font color='green'>***Exercise 0.3: As always, please compile your own program in a compact way, where you can deal with different input data for training, and also staying flexible with model configuration. Please use the .csv file containing multiple bands and corresponding labels to test your model performance on different input bands.***</font>

*Start to load training data:*

```
import pandas as pd
lulc = pd.read_csv('uu_ml/data/stack_aoi_2015.csv')

# View some sample rows of the data
lulc.head()
```

*You are also encourage to create your own AOI labels, load it, rasterize it and concantenate it together with the images:*

```
aoi = gpd.read_file('...')

rst = rasterio.open('...')  # Base image to rasterize the *.shp
meta = rst.meta.copy()  # Copy metadata from the base image
meta.update(compress='lzw')

...
```

*Train your model and inspect the training processes*

```
from sklearn... import ...
    ...
    ...
tree_depth = ##    
band_list = [##, ##, ..., ##]
    
```

*Evaluate the accuracy with different metrics*

```
...
...
```

*Generalize your prediction from the trained model to larger areas*

```
...
...
```